<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/parsing3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
sys.path.append('/content/') #to be able for importing libarary(drain)from this address
import Drain
import pandas as pd
import numpy as np


# Upload dataset

In [3]:
!wget 'https://zenodo.org/record/3227177/files/HDFS_1.tar.gz'

--2023-07-21 15:46:19--  https://zenodo.org/record/3227177/files/HDFS_1.tar.gz
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 161886385 (154M) [application/octet-stream]
Saving to: ‘HDFS_1.tar.gz’

HDFS_1.tar.gz       100%[===================>] 154.39M   420KB/s    in 6m 23s  

2023-07-21 15:52:43 (413 KB/s) - ‘HDFS_1.tar.gz’ saved [161886385/161886385]



In [4]:
!tar -xzvf "/content/HDFS_1.tar.gz" -C "/content/"   #unzip the file

HDFS.log
anomaly_label.csv


In [5]:
# count rows of the raw log

with open('/content/HDFS.log', "r") as file:
    totaln=0
    for line in file:
        totaln += 1

print('There are a total of {} lines'.format(totaln))

There are a total of 11175629 lines


In [6]:
#Take a quick look at the first lines of the raw log:

with open('/content/HDFS.log', "r") as file:
  n = 0
  for line in file:
    print(line)
    n += 1
    if n == 5:
      break

081109 203518 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010

081109 203518 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.allocateBlock: /mnt/hadoop/mapred/system/job_200811092030_0001/job.jar. blk_-1608999687919862906

081109 203519 143 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.10.6:40524 dest: /10.250.10.6:50010

081109 203519 145 INFO dfs.DataNode$DataXceiver: Receiving block blk_-1608999687919862906 src: /10.250.14.224:42420 dest: /10.250.14.224:50010

081109 203519 145 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_-1608999687919862906 terminating



In [ ]:
#Take a quick look at the labels, which we see, its classified based on blockID

labels = pd.read_csv('/content/anomaly_label.csv')
labels.head()

,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly
3,blk_-9073992586687739851,Normal
4,blk_7854771516489510256,Normal


In [ ]:
len(labels)

575061

# Parsing

In [ ]:
input_dir  = '/content/'  # The input directory of log file
output_dir = 'Drain_result/'  # The output directory of parsing results
log_file   = 'HDFS.log'  # The input log file name
log_format = "<Date> <Time> <Pid> <Level> <Component>: <Content>"  # HDFS log format
# Regular expression list for optional preprocessing (default: [])
regex      = [
    r'blk_(|-)[0-9]+' , # block id
    r'(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)', # IP
    r'(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$', # Numbers
]
st         = 0.5  # Similarity threshold
depth      = 4  # Depth of all leaf nodes

parser = Drain.LogParser(log_format, indir=input_dir, outdir=output_dir,  depth=depth, st=st, rex=regex)
parser.parse(log_file)

Parsing file: /content/HDFS.log
Total size after encoding is 11175629 11175629
Parsing done. [Time taken: 0:26:52.392421]


In [ ]:

import os
import re
import numpy as np
import pandas as pd
from collections import OrderedDict

struct_log = pd.read_csv('/content/Drain_result/HDFS.log_structured.csv', engine='c', na_filter=False, memory_map=True)



In [ ]:
struct_log.head()

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,[]
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,d38aa58d,PacketResponder <*> for block <*> <*>,[]


In [ ]:
struct_log.head()

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,[]
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,d38aa58d,PacketResponder <*> for block <*> <*>,[]


In [ ]:
# we assume that we start again the code from here and we have to upload the structured log again(fast)
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
struct_log = pd.read_csv('/content/drive/MyDrive/pasrsing', engine='c', na_filter=False, memory_map=True)

In [ ]:
struct_log.head()

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,3d91fa85,BLOCK* NameSystem.allocateBlock: <*> <*>,[]
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,09a53393,Receiving block <*> src: <*> dest: <*>,[]
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,d38aa58d,PacketResponder <*> for block <*> <*>,[]


In [ ]:
# log_file is too heavy so we just take first half milion lines
struct_log2 = struct_log[:500000]

In [ ]:
# Get the unique items of the 'EventId' column
unique_EventId = struct_log2['EventId'].unique()

print(unique_EventId)

['09a53393' '3d91fa85' 'd38aa58d' 'e3df2680' '5d5de21c' 'd6b7b743'
 '73c2ec69' '75627efd' '54e5f6b4' '626085d5' '728076ac' '40651754'
 '32777b38' 'ace40671' '506b3c4b' 'bcc910df' 'dba996ef' '0567184d'
 '69bca6e5' 'd013b7a3' 'c294d20f' 'f0d1ff15' '2f85639c' 'f266840a'
 'b15ffff8' '461a1574' '71cf10b1' 'c61b491b' '2ecc047e' '4610d0f1'
 '68429f34' 'd63ef163' '2e68ccc3' '8f2bc724' '81cee340']


In [ ]:

# Define the mapping dictionary automatically
mapping = {item: index for index, item in enumerate(unique_EventId)}

print(mapping)


{'09a53393': 0, '3d91fa85': 1, 'd38aa58d': 2, 'e3df2680': 3, '5d5de21c': 4, 'd6b7b743': 5, '73c2ec69': 6, '75627efd': 7, '54e5f6b4': 8, '626085d5': 9, '728076ac': 10, '40651754': 11, '32777b38': 12, 'ace40671': 13, '506b3c4b': 14, 'bcc910df': 15, 'dba996ef': 16, '0567184d': 17, '69bca6e5': 18, 'd013b7a3': 19, 'c294d20f': 20, 'f0d1ff15': 21, '2f85639c': 22, 'f266840a': 23, 'b15ffff8': 24, '461a1574': 25, '71cf10b1': 26, 'c61b491b': 27, '2ecc047e': 28, '4610d0f1': 29, '68429f34': 30, 'd63ef163': 31, '2e68ccc3': 32, '8f2bc724': 33, '81cee340': 34}


In [ ]:
# Replace the values using the mapping dictionary
struct_log2['EventId'] = struct_log2['EventId'].replace(mapping)


<ipython-input-7-e634dd0945ad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  struct_log2['EventId'] = struct_log2['EventId'].replace(mapping)


In [ ]:
struct_log2.head()

,LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate,ParameterList
0,1,81109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,0,Receiving block <*> src: <*> dest: <*>,[]
1,2,81109,203518,35,INFO,dfs.FSNamesystem,BLOCK* NameSystem.allocateBlock: /mnt/hadoop/m...,1,BLOCK* NameSystem.allocateBlock: <*> <*>,[]
2,3,81109,203519,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,0,Receiving block <*> src: <*> dest: <*>,[]
3,4,81109,203519,145,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: ...,0,Receiving block <*> src: <*> dest: <*>,[]
4,5,81109,203519,145,INFO,dfs.DataNode$PacketResponder,PacketResponder 1 for block blk_-1608999687919...,2,PacketResponder <*> for block <*> <*>,[]


In [ ]:
import re
from collections import OrderedDict

data_dict = OrderedDict()  # to search an iten faster
for idx, row in struct_log2.iterrows():
    blkId_list = re.findall(r'(blk_-?\d+)', row['Content'])
    blkId_set = set(blkId_list)
    for blk_Id in blkId_set:
        if not blk_Id in data_dict:
            data_dict[blk_Id] = []
        data_dict[blk_Id].append(row['EventId'])
data_df = pd.DataFrame(list(data_dict.items()), columns=['BlockId', 'EventSequence'])
data_df.to_csv("HDFS_sequence.csv",index=None)


In [ ]:
data_df

,BlockId,EventSequence
0,blk_-1608999687919862906,"[0, 1, 0, 0, 2, 2, 3, 3, 2, 3, 4, 4, 4, 5, 0, ..."
1,blk_7503483334202473044,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 9, 12,..."
2,blk_-3544583377289625738,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 9, 4, 4, 4, 9, ..."
3,blk_-9073992586687739851,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12..."
4,blk_7854771516489510256,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12..."
...,...,...
36300,blk_-5820341108159367267,"[1, 8, 0, 2, 3, 0, 2, 3, 0, 2, 3, 4, 4, 4, 7, ..."
36301,blk_-6470779133565377044,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4, 9]"
36302,blk_-7724713468912166542,"[0, 2, 3, 9, 0, 2, 3, 0, 2, 3, 4, 1, 4, 4, 9, ..."
36303,blk_-4615135864434101729,"[0, 2, 3, 0, 2, 3, 4, 4, 1, 4, 0, 2, 3]"


In [ ]:
labels = pd.read_csv('/content/drive/MyDrive/anomaly_label.csv')
labels.head()

,BlockId,Label
0,blk_-1608999687919862906,Normal
1,blk_7503483334202473044,Normal
2,blk_-3544583377289625738,Anomaly
3,blk_-9073992586687739851,Normal
4,blk_7854771516489510256,Normal


In [ ]:
# Merge the labels with the data_df DataFrame
data_df = data_df.merge(labels, on='BlockId', how='left')
data_df.head()

,BlockId,EventSequence,Label
0,blk_-1608999687919862906,"[0, 1, 0, 0, 2, 2, 3, 3, 2, 3, 4, 4, 4, 5, 0, ...",Normal
1,blk_7503483334202473044,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 9, 12,...",Normal
2,blk_-3544583377289625738,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 9, 4, 4, 4, 9, ...",Anomaly
3,blk_-9073992586687739851,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12...",Normal
4,blk_7854771516489510256,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12...",Normal


In [ ]:
# Splitting the dataset into train and test
normal_data_df = data_df[data_df['Label'] == 'Normal']
anomaly_data_df = data_df[data_df['Label'] != 'Normal']

train_ratio = 0.8  # 80% for training
train_normal_len = int(len(normal_data_df) * train_ratio)

train_data_df = normal_data_df[:train_normal_len]
test_data_df = pd.concat([normal_data_df[train_normal_len:], anomaly_data_df])

# Print the lengths of train and test datasets
print("Train Dataset Length:", len(train_data_df))
print("Test Dataset Length:", len(test_data_df))

Train Dataset Length: 27646
Test Dataset Length: 8659


In [ ]:
train_data_df

,BlockId,EventSequence,Label
0,blk_-1608999687919862906,"[0, 1, 0, 0, 2, 2, 3, 3, 2, 3, 4, 4, 4, 5, 0, ...",Normal
1,blk_7503483334202473044,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 9, 12,...",Normal
3,blk_-9073992586687739851,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12...",Normal
4,blk_7854771516489510256,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12...",Normal
5,blk_1717858812220360316,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12, 12...",Normal
...,...,...,...
29007,blk_4161999198115942018,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Normal
29008,blk_9045586653678877558,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Normal
29010,blk_7576088765244670114,"[0, 0, 1, 0, 2, 3, 2, 3, 3, 2, 4, 4, 4]",Normal
29011,blk_-8231801651473885756,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Normal


In [ ]:
test_data_df

,BlockId,EventSequence,Label
29013,blk_8535110522048708059,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Normal
29014,blk_-40031802254835012,"[0, 0, 0, 1, 2, 3, 2, 3, 4, 2, 3, 4, 4, 34]",Normal
29015,blk_4155080411728318342,"[0, 0, 1, 0, 2, 3, 4, 2, 3, 2, 3, 4, 4]",Normal
29016,blk_7818924753936381970,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4, 12]",Normal
29017,blk_-1445048717348719652,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Normal
...,...,...,...
36220,blk_8388642701725277995,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Anomaly
36232,blk_9056514806795308459,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4, 31, 31...",Anomaly
36279,blk_5659869911521029625,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4, 31, 31...",Anomaly
36296,blk_-6067429461406356955,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4, 31, 31...",Anomaly


In [ ]:
# shuffle the DataFrame rows
test_data_df = test_data_df.sample(frac = 1)
test_data_df

,BlockId,EventSequence,Label
34916,blk_-5001730185386665931,"[0, 0, 0, 1, 3, 2, 3, 2, 3, 2, 4, 4, 4]",Normal
30921,blk_-5112935196433124464,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Normal
33105,blk_-3782424069271015476,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Normal
34291,blk_4331908968666738601,"[0, 0, 0, 1, 2, 3, 2, 3, 2, 3, 4, 4, 4, 34, 9]",Normal
31625,blk_-4541842725664362663,"[0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Normal
...,...,...,...
17182,blk_8227133095928812750,"[0, 1, 0, 0, 2, 3, 2, 3, 4, 4, 4, 2, 3]",Anomaly
29229,blk_2022192046743126895,"[0, 0, 1, 0, 2, 3, 4, 2, 3, 2, 3, 4, 4]",Normal
29608,blk_-2588002185777385617,"[0, 0, 0, 1, 2, 2, 3, 3, 4, 4, 2, 3, 4]",Normal
32036,blk_-6751276971729439154,"[0, 1, 0, 0, 2, 3, 2, 3, 2, 3, 4, 4, 4]",Normal


In [ ]:
# save datasets on google drive
data_df.to_csv('/content/drive/MyDrive/data_df.csv')
train_data_df.to_csv('/content/drive/MyDrive/train_data_df.csv')
test_data_df.to_csv('/content/drive/MyDrive/test_data_df.csv')